In [1]:
import pandas as pd
import re

***src***

In [2]:
def basic_cleaning():
    #modificamos todas las letras a minúsculas
    df.applymap(lambda x: x.lower() if type(x) == str else x)
    #modificamos los espacios blancos en los titulos de las las columnas por añadimos _
    df.columns = [i.lower().replace(" ", "_") for i in df.columns]
    #eliminamos duplicados
    df.drop_duplicates()
    return df


In [3]:
def cleaning_df1():
    #renombramos la columna species yhacemos una copia
    df1.rename(columns={'species_': 'species'}, inplace=True)
    df1['species2']=df1['species'].copy()
    #eliminamos todas las filas de las columnas indicadas con Nans.
    df1.dropna(subset=['date', 'type', 'country', 'area', 'location', 'activity', 'sex_', 'species2'], how='all', inplace=True)
    #eliminamos las columnas indicadas del df1
    df1.drop(['case_number', 'date','year', 'unnamed:_23', 'unnamed:_22', 'case_number.2', 'case_number.1', 'href', 'href_formula', 'pdf', 'investigator_or_source', 'time'], axis=1, inplace=True)

    
    return df1

In [4]:
def cleaning_df2():

    #eliminamos las filas con valores Nan en species
    df2.dropna(subset=['species2'], inplace=True)
    #deletting anything that's not a letter
    df2['species2'].apply(lambda x: re.sub(r'\W+', ' ', x))
    #It searches for any numeric character (\d) followed by an optional character (.) and removes it
    df2['species2'].apply(lambda x: re.sub(r'\d.?', '', x, flags=re.IGNORECASE))
    
    
    return df2

In [20]:
def species_cleaning():
    species_values=['white', 'nurse', 'blacktip', 'tiger', 'wobbegong', 'bull', 'blacktip', 'hammerhead', 'Mako', 'lemon', 'blacktip or spinner shark', 'largeako', 'oceanic whitetip', 'bronze whaler', 'spinner', 'caribbean reef', 'blue pointer', 'blue shark', 'bull shark','brown shark','zambezi shark', 'unidentified' ]
    not_involved= ['Shark involvement prior to death unconfirmed', 'Shark involvement prior to death not confirmed', 'Shark involvement suspected but not confirmed', 'Invalid']
    
    #crear un diccionario con species_values
    
    
    #ELIMINAR LAS FILAS QUE EN TYPE PONGA INVALID
    df2.drop(df2[df2['type'] =='invalid'].index, inplace=True)
    
    for case in not_involved:
        df2.drop(df2[df2['species2'] == case].index, inplace=True)

    #modificamos las celdas donde aparece una especie
    for specie in species_values:
        df2.loc[(df2['species2'].str.contains(specie, case=False)), 'species2'] = specie
        
    #elimina las letras m y a si se encuentran fuera de palabras
    df2['species2'] = df2['species2'].str.replace(r'\s[m|a]\s', ' ', regex=True)
    df2['species2'] = df2['species2'].str.replace(r'^m|a\s', '', regex=True)
    df2['species2'] = df2['species2'].str.replace(r'\sm|as$', '', regex=True)
    
    #eliminamos las palabras con dos letras de toda la columna
    df2['species2'] = df2['species2'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))
    
    
    return df2

***DATAFRAME***

In [6]:
df=pd.read_csv('data/attacks.csv', encoding='latin')

In [7]:
df= basic_cleaning()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_number             8702 non-null   object 
 1   date                    6302 non-null   object 
 2   year                    6300 non-null   float64
 3   type                    6298 non-null   object 
 4   country                 6252 non-null   object 
 5   area                    5847 non-null   object 
 6   location                5762 non-null   object 
 7   activity                5758 non-null   object 
 8   name                    6092 non-null   object 
 9   sex_                    5737 non-null   object 
 10  age                     3471 non-null   object 
 11  injury                  6274 non-null   object 
 12  fatal_(y/n)             5763 non-null   object 
 13  time                    2948 non-null   object 
 14  species_                3464 non-null 

In [8]:
df.sample()


,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
13532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
 #realizamos una copia del df principal y creamos df1
df1= df.copy()    

In [10]:
df1= cleaning_df1()

In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6302 entries, 0 to 6301
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   type            6298 non-null   object 
 1   country         6252 non-null   object 
 2   area            5847 non-null   object 
 3   location        5762 non-null   object 
 4   activity        5758 non-null   object 
 5   name            6092 non-null   object 
 6   sex_            5737 non-null   object 
 7   age             3471 non-null   object 
 8   injury          6274 non-null   object 
 9   fatal_(y/n)     5763 non-null   object 
 10  species         3464 non-null   object 
 11  original_order  6302 non-null   float64
 12  species2        3464 non-null   object 
dtypes: float64(1), object(12)
memory usage: 689.3+ KB


In [12]:
df1.sample()

,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),species,original_order,species2
5500,Invalid,AUSTRALIA,Victoria,Elwood,Bathing,Charles Blake,M,23,Thought to have been taken by a shark,NaN,Questionable,803.0,Questionable


In [13]:
#creamos una copia del dataframe: df2
df2=df1.copy()

In [14]:
df2=cleaning_df2()

In [15]:
df2.sample()

,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),species,original_order,species2
525,Invalid,NaN,NaN,NaN,Sea disaster,Cuban refugees,M,NaN,Shark involvement prior to death not confirmed,NaN,Shark involvement not confirmed,5778.0,Shark involvement not confirmed


In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3464 entries, 0 to 6296
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   type            3464 non-null   object 
 1   country         3452 non-null   object 
 2   area            3303 non-null   object 
 3   location        3288 non-null   object 
 4   activity        3235 non-null   object 
 5   name            3367 non-null   object 
 6   sex_            3145 non-null   object 
 7   age             2193 non-null   object 
 8   injury          3453 non-null   object 
 9   fatal_(y/n)     2948 non-null   object 
 10  species         3464 non-null   object 
 11  original_order  3464 non-null   float64
 12  species2        3464 non-null   object 
dtypes: float64(1), object(12)
memory usage: 378.9+ KB


In [21]:
df2=species_cleaning()

In [22]:
df2

,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),species,original_order,species2
0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,White shark,6303.0,white
3,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,2 m shark,6300.0,shark
4,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m",6299.0,tiger
6,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Tiger shark,6297.0,tiger
7,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,"Lemon shark, 3'",6296.0,lemon
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6276,Unprovoked,BELIZE,NaN,NaN,Standing,a servant,M,16,FATAL,Y,12' tiger shark,27.0,tiger
6293,Unprovoked,AUSTRALIA,NaN,NaN,Fishing,boy,M,NaN,"FATAL, knocked overboard by tail of shark & ca...",Y,Blue pointer,10.0,blue pointer
6294,Unprovoked,AUSTRALIA,NaN,NaN,Fishing,fisherman,M,NaN,FATAL,Y,Blue pointer,9.0,blue pointer
6295,Unprovoked,AUSTRALIA,NaN,NaN,Fishing,fisherman,M,NaN,FATAL,Y,Blue pointers,8.0,blue pointer


In [23]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3277 entries, 0 to 6296
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   type            3277 non-null   object 
 1   country         3266 non-null   object 
 2   area            3138 non-null   object 
 3   location        3121 non-null   object 
 4   activity        3095 non-null   object 
 5   name            3205 non-null   object 
 6   sex_            2987 non-null   object 
 7   age             2133 non-null   object 
 8   injury          3267 non-null   object 
 9   fatal_(y/n)     2946 non-null   object 
 10  species         3277 non-null   object 
 11  original_order  3277 non-null   float64
 12  species2        3277 non-null   object 
dtypes: float64(1), object(12)
memory usage: 358.4+ KB


In [24]:
Species_unique2= []
for i in df2['species2'].unique():
    Species_unique2.append(i)
print(Species_unique2)

['white', 'shark', 'tiger', 'lemon', 'bull', 'Grey reef shark', 'Invalid incident', 'nurse', 'Shark involvement not confirmed', 'Questionable', 'wobbegong', '3.5 shark', '1.8 shark', 'blacktip', 'Injury believed caused eel, not shark', 'Galapagos shark?', 'Grey reef shark.', 'small shark', 'blue shark', 'Cookiecutter shark', 'spinner', 'Deathay have been due drowning', 'Porbeagle, 1.5', 'Shark involvement questionable', 'small shark"', 'shark invovlement', 'Seven-gill shark', 'shark, probably smooth hound', 'Shark involvement highly doubtful', 'shark invovlement publicity stunt', "10' shark", 'caribbean reef', 'Broadnose sevengill shark, 1.5', 'reef shark,', 'Angel shark', 'dogfish shark', 'Said involve shark butore likely damage caused debris', 'Mako', 'bronze whaler', 'Grey reef shark,', 'shark involvement', 'Reef shark,', "Silky shark, 6.5'", 'Galapagos shark', 'hammerhead', 'juvenile shark', 'shark?', 'Thought involve shark, but shark involvement not confirmed', 'Shark involvement 

In [25]:
(len(Species_unique2))

503